# Introducción a la Visualización de Redes

En este notebook aprenderemos a visualizar redes con el módulo `aves` a través de [graphtool](http://graph-tool.skewed.de/), una biblioteca para trabajar con redes en Python.

Los propósitos de esta clase son los siguientes:

  1. Cargar una red.
  2. Explorar como visualizarla.
  3. Definir una tarea y usar visualización para responderla.
  
Como data set de prueba utilizaremos la Red de Jazz que se encuentra en [Colección de Redes de Koblenz](http://konect.uni-koblenz.de/networks/arenas-jazz). Ya está incluida en el repositorio `aves`, en la carpeta `data/external/arenas-jazz`.


## Preámbulo y Carga de Datos

In [ ]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

load_dotenv()

AVES_ROOT = Path(os.environ['AVES_ROOT'])

DATA_PATH = AVES_ROOT / "data" / "external" / "arenas-jazz"
DATA_PATH

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from aves.config import setup_style

setup_style()

El dataset que utilizaremos tiene un archivo README que explica su origen y composición. Está en inglés.

In [ ]:
!cat {DATA_PATH}/README.arenas-jazz

En castellano, dice:

> Esta es una red de colaboración entre músicos y músicas de Jazz. Cada nodo es una persona y una arista implica que dos personas han interpretado música juntes en una banda. Los datos fueron recolectados el año 2003. 

Veamos la cabecera del archivo:

In [ ]:
!head {DATA_PATH}/out.arenas-jazz -n 10

El archivo que contiene la red tiene dos tipos de líneas:

- La primera, que tiene un comentario. Podemos ignorarlo.
- El resto, donde cada línea representa una arista. En ella aparecen dos números: el identificador del nodo de origen de una arista, y el identificador del nodo de destino de la misma arista.

Podemos usar `pandas` para cargar el archivo sin problemas.

In [ ]:
edgelist = pd.read_csv(
    # ruta al archivo
    f"{DATA_PATH}/out.arenas-jazz",
    # noten que está separado por TABS, no por comas
    sep="\t",
    # no leer la primera línea
    skiprows=1,
    # no tiene nombres de columnas
    header=0,
    # estos son los nombres de las columnas
    names=["source", "target"],
)

edgelist


En aves la clase `Network` nos permite crear una red a partir de una tabla de aristas. Notemos que esta es una red no dirigida, es decir, las aristas no tienen dirección, se pueden interpretar en cualquier dirección: si A colabora con B, B también colabora con A.

In [ ]:
from aves.models.network import Network

network = Network.from_edgelist(edgelist, directed=False)
network.num_vertices, network.num_edges


La visualización `NodeLink` se construye a partir del modelo `Network`:

In [9]:
from aves.visualization.networks import NodeLink

# creación
nodelink = NodeLink(network)

# organización de los noodos
nodelink.layout_nodes()

# el tamaño de un nodo será proporcional a la cantidad de conexiones que tenga
nodelink.set_node_drawing(method="plain", weights="total_degree")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# el tamaño del nodo es proporcinoal a las conexiones.
# el parámetro node_size es el tamaño máximo.
nodelink.plot(
    ax,
    nodes=dict(node_size=150, facecolor="white", edgecolor="black", linewidth=1),
    edges=dict(alpha=0.5),
)

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()

# fig.savefig('../reports/figures/example_nodelink.png')

También podemos configurar la apariencia de las aristas. Ya que no son dirigidas ni tienen peso, debemos asignarles uno. Una manera común de hacerlo es a través de la [centralidad](https://en.wikipedia.org/wiki/Betweenness_centrality):

In [11]:
nodelink.set_edge_drawing(method="weighted", weights='betweenness', k=20)


In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

nodelink.plot(ax, nodes=dict(node_size=100, facecolor='white', edgecolor="black", linewidth=1), edges=dict(palette='Purples'))

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()


Pongamos atención a lo siguiente:

1. Cuando cargamos la red, en ningún momento especificamos una posición `(x, y)` para cada nodo de la red.
2. Al visualizar la red, esas posiciones tienen que salir de algún método.
  
El método que genera las posiciones es un _algoritmo de organización_ (_layout algorithm_). Estos algoritmos posicionan los nodos en la imagen, y algunos utilizan números aleatorios para encontrar buenas posiciones de los nodos. Al ser un proceso que se ejecuta cada vez que dibujamos la red, el resultado es distinto. 

Distinto, pero _similar_: el algoritmo tiende a hacer lo mismo.

Primero probamos el método `sfdp`. Ahora veamos el método `arf`:

In [13]:
nodelink.layout_nodes(method="force-directed", algorithm="arf")


In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

nodelink.plot(ax, nodes=dict(node_size=150, facecolor="white", edgecolor="black", linewidth=1, alpha=0.9))

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()


## Estimar y Visualizar Caminos en la Red

Una tarea común es **visualizar los caminos en una red**. Por ejemplo, cada vez que pides instrucciones o caminos en una aplicación de mapas, estás observando el resultado de una estimación de caminos en la red (usualmente el camino más corto, _shortest path_). La diferencia es que en los mapas el territorio define la organización.

En el caso de una red de colaboración, una posible interpretación de un camino más corto es la secuencia de personas a las que debes contactar para conocer a alguien específico.

Veamos como graficar caminos entre dos nodos aleatorios en la red.
Para ello, importaremos el módulo `random`:

In [ ]:
import random 

src = random.randint(1, network.num_vertices)
dst = random.randint(1, network.num_vertices)

src, dst

El método `shortest_path` entrega la secuencia de nodos para ir desde un nodo origen hasta un nodo destino:

In [ ]:
shortest_paths = network.shortest_path(src, dst)
shortest_paths

Para graficar un camino (el primero que se haya encontrado) crearemos una vista parcial de la red que solo contenga los nodos correspondientes:

In [ ]:
view = network.subgraph(shortest_paths[0])
view.num_vertices, view.num_edges

In [18]:
nodelink_view = NodeLink(view)
nodelink_view.set_edge_drawing(method="plain")

In [19]:
nodelink_view.set_node_drawing(method='labeled')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# full network
nodelink.plot(ax, nodes=dict(node_size=50, color="white", edgecolor="black"))

# shortest path with highlights
nodelink_view.plot(
    ax,
    edges=dict(color="orange", linewidth=2),
    nodes=dict(node_size=120, color="purple", edgecolor="black"),
)

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()


In [21]:
network.detect_communities()

In [22]:
nodelink.set_node_drawing(
    # peso de los nodos
    weights='total_degree',
    # categorías a utilizar para colorear los nodos
    categories='community',
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

nodelink.plot(ax, nodes=dict(node_size=150, facecolor="white", edgecolor="black", linewidth=1, alpha=0.9, palette='husl'))

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()

Como vemos, tenemos una noción de dónde está cada uno de los nodos involucrados en la red, e incluso podemos ver que al parecer el camino cruza desde una comunidad a otra.

Ahora bien, debemos tener cuidado al interpretar una visualización como ésta: el algoritmo de organización tiene un criterio gráfico para posicionar nodos en la imagen, una comunidad o _cluster_ en la visualización no necesariamente representa un _cluster real en la estructura_.

## Tarea: Detección de Comunidades

Una tarea común en el análisis de redes es encontrar comunidades de nodos, es decir, grupos que estén altamente conectados entre sí en comparación con las conexiones de los demás nodos. A veces estamos interesados en el comportamiento colectivo de la red, y en ese caso, podríamos analizar los patrones de comportamiento de las comunidades en vez de los de cada nodo.

En nuestro data set, la visualización sugiere que hay al menos dos comunidades. Quizás hay más. No lo sabemos aún. Pero, ¿cuál es el significado de comunidad aquí? El data set es sobre músicos de jazz que interpretan música juntos. Entonces, quizás cada comunidad se refiere a estilos específicos de interpretación, a variaciones del jazz, a ubicaciones en las que tocan música, etc.


Los nodos que están en los bordes del círculo son los músicos, y los nodos interiores son las comunidades.

Noten como los nodos están ordenados de acuerdo a las comunidades. El nodo central es una raíz artificial que agregamos en nuestra magia negra, que nos permitió darle la estructura de árbol a la red total.

El siguiente paso es generar el diccionario de posiciones solamente para los nodos de nuestra red original:

In [24]:
network.detect_communities(method='hierarchical')

In [ ]:
from cytoolz import valmap
import numpy as np
valmap(lambda x: np.unique(x), network.communities_per_level)

In [ ]:
network.set_community_level(0)
nodelink.set_node_drawing(
    "plain",
    weights="total_degree",
    # categorías a utilizar para colorear los nodos. pueden ser las mismas de HEB
    categories='community',
)

fig, ax = plt.subplots(figsize=(6, 6))

nodelink.plot(ax, 
    nodes=dict(node_size=150, palette='plasma', edgecolor='none', alpha=0.75), 
    edges=dict(color='#abacab', alpha=0.5))


ax.set_axis_off()
ax.set_aspect('equal')
fig.tight_layout()

In [27]:
heb = nodelink.bundle_edges(method="hierarchical")

In [28]:
network.set_community_level(2)
nodelink.set_node_drawing(
    # muestra las etiquetas
    "labeled",
    # como el método HEB es radial, podemos aprovechar eso para orientar el texto (rotarlo)
    radial=True,
    # distancia radial desde los nodos
    offset=0.1,
    # peso de los nodos
    weights="total_degree",
    # categorías a utilizar para colorear los nodos. pueden ser las mismas de HEB
    categories='community',
)


In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

nodelink.plot(ax, 
    nodes=dict(node_size=150, palette='plasma', edgecolor='none', alpha=0.75, fontsize='x-small'), 
    edges=dict(color='#abacab', alpha=0.5))


ax.set_axis_off()
ax.set_aspect('equal')
fig.tight_layout()

Podemos visualizar la estructura de HEB para entender lo que hace este método:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

nodelink.plot(
    ax,
    nodes=dict(
        node_size=5, palette="plasma", edgecolor="none", alpha=0.75),
    edges=dict(color="#abacab", alpha=0.5),
)

heb.plot_community_network(ax)

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()


Increíble, ¿no? Ahora podemos ver como las comunidades se conectan entre sí y también como los miembros de una comunidad se relacionan. Vemos quienes son populares y quienes son puentes con otras comunidades. Piensen en lo útil que podría ser esto para un agente artístico.

Podemos utilizar la categorización para colorear las aristas:

In [31]:
network.set_community_level(2)
nodelink.set_node_drawing(
    # muestra las etiquetas
    "labeled",
    # como el método HEB es radial, podemos aprovechar eso para orientar el texto (rotarlo)
    radial=True,
    # distancia radial desde los nodos
    offset=0.1,
    # peso de los nodos
    weights='pagerank',
    # categorías a utilizar para colorear los nodos. pueden ser las mismas de HEB
    categories='community',
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

nodelink.plot(ax, 
    nodes=dict(node_size=150, palette='plasma', edgecolor='none', alpha=0.75, fontsize='x-small'), 
    edges=dict(color='#abacab', alpha=0.5))


ax.set_axis_off()
ax.set_aspect('equal')
fig.tight_layout()

In [33]:
nodelink.set_edge_drawing(
    "community-gradient", level=2
)


In [34]:
nodelink.set_node_drawing(
    # muestra las etiquetas
    "plain",
    # como el método HEB es radial, podemos aprovechar eso para orientar el texto (rotarlo)
    radial=True,
    # distancia radial desde los nodos
    offset=0.1,
    # peso de los nodos
    weights="total_degree",
    # categorías a utilizar para colorear los nodos. pueden ser las mismas de HEB
    categories='community',
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

# necesitamos graficar los nodos para ajustar los límites de la visualización.
nodelink.plot_nodes(ax, alpha=0)

nodelink.plot_edges(ax, alpha=0.5)

# aquí podríamos tener una función que tome el identificador de la comunidad y 
# genere un texto para desplegar como nombre.
nodelink.bundle_model.plot_community_wedges(
    ax, wedge_width=0.05, level=1, wedge_offset=0.01, label_func=lambda x: x, palette="turbo"
)

ax.set_axis_off()
ax.set_aspect("equal")
fig.tight_layout()

# fig.savefig('../reports/figures/example_heb.png')


Ahora bien, en Hierarchical Edge Bundling las comunidades también dependen de la inicialización del algoritmo, que utiliza números generados aleatoriamente. En un análisis riguroso, el procedimiento es repetir el proceso varias veces (decenas o incluso cientos de veces), y luego utilizar un método que permita saber cuál es el modelo que presenta mejor ajuste. En el caso de `graph_tool`, esto se hace [eligiendo el modelo que genera la descripción más compacta de la red](https://graph-tool.skewed.de/static/doc/demos/inference/inference.html#the-stochastic-block-model-sbm).

Con esto terminamos esta clase.

La visualización de redes permite estudiar los datos a traves de las relaciones presentes en ellos, lo que la vuelve un buen complemento para las otras técnicas que hemos trabajado en el curso.

## Propuesto

Ya que la detección de comunidades jerárquica crea un árbol, ¿es posible usar un `treemap` para visualizar la red?